# <font color='Black'>Webscraping no </font> <font color='RED'> Youtube </font>

### <font color='Grey'>Criando Máquinas Preditivas por meio de DADOS extraídos de Redes Sociais - PLN e ML</font>

#1° Problema de Negócio


### Precisamos Criar uma Máquina Preditiva para **descobrir qual a categoria de um vídeo postado no Youtube por meio da Análise do seu Título.**

#2° Análise Exploratória dos Dados


##2° Faça o Download da API "**chromedriver**" de acordo com a respectiva a versão do seu Google Chrome, conforme abaixo:
![](https://miro.medium.com/max/3750/1*2EFPAOgwteeRQcJC2JZMlw.png)

##3° Vá até a pasta de downloads e **clique com o botão direito e descompacte** o arquivo selecionando a **opção Extrair aqui.**

##4° Abra o "Terminal" (nosso cmd do linux) e navegue até a pasta de "Downloads", digitando o comando abaixo:

cd /home/~seu-usuario~/Downloads

~seu-usuario~ aqui é o usuário que vc criou quando estava construindo a VM, hein? **Ai ai....eu te pego....**




![](https://cienciadosdados.com/images/2021/linux_down.png)

##5° Digite a linha de comando abaixo:

sudo mv chomedriver /usr/local/bin/




![](https://cienciadosdados.com/images/2021/linux.png)

In [ ]:
# Instalação do Pacote para Webscraping
!pip install selenium

In [ ]:
# Mapeamento de Pacotes
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Escolhe a categoria Travel
driver = webdriver.Chrome() 
driver.get("https://www.youtube.com/results?search_query=travel")

In [ ]:
# Busca do nome do Vídeo por meio do Xpath
user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
links = []
for i in user_data:
            links.append(i.get_attribute('href'))

print(len(links))

In [ ]:
# Criação da Tabela que vai receber os Vídeos da categoria TRAVEL
df_travel = pd.DataFrame(columns = ['link', 'title', 'category'])

In [ ]:
# Fazendo o Scraping dos Dados
wait = WebDriverWait(driver, 10)
category = "travel"
for x in links:
            driver.get(x)
            id = x.strip('https://www.youtube.com/watch?v=')
            title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text
            df_travel.loc[len(df_travel)] = [id, title, category]

In [ ]:
# Printando a Tabela
df_travel

In [ ]:
# Escolhe a categoria science
driver = webdriver.Chrome() 
driver.get("https://www.youtube.com/results?search_query=science")

In [ ]:
# Busca do nome do Vídeo por meio do Xpath
user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
links = []
for i in user_data:
            links.append(i.get_attribute('href'))

print(len(links))

In [ ]:
# Criação da Tabela que vai receber os Vídeos da categoria TRAVEL
df_science = pd.DataFrame(columns = ['link', 'title', 'category'])

In [ ]:
# Fazendo o Scraping dos Dados
wait = WebDriverWait(driver, 10)
category = "science"
for x in links:
            driver.get(x)
            id = x.strip('https://www.youtube.com/watch?v=')
            title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text
            df_science.loc[len(df_science)] = [id, title, category]

In [ ]:
# Printando a Tabela
df_science

In [ ]:
# Concatenando as Tabelas das categorias 
frames = [df_travel, df_science]
df_copy = pd.concat(frames, axis=0, join='outer',  ignore_index=True,
                            keys=None, levels=None, names=None, verify_integrity=False, copy=True)

In [ ]:
# Printando a Tabela
df_copy

In [ ]:
#Criando dataframes vazios
df_link = pd.DataFrame(columns = ["link"])        
df_title = pd.DataFrame(columns = ["title"])        
df_category = pd.DataFrame(columns = ["category"])        


In [ ]:
df_link

In [ ]:
#Preenchendo os dataframes
df_link['link'] = df_copy['link'] 
df_title ['title']= df_copy['title'] 
df_category['category'] = df_copy['category']

In [ ]:
#Verificando as dimensões da tabela
df_title.shape

#3° Pré-Processamento dos Dados

### **PLN - Processamento de Linguagem Natural**

In [ ]:
# Importando Pacotes de PLN
import re 
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

In [ ]:
#Realizando a Mineração do Texto 
corpus = []        
for i in range(0, 52):         
  
  review = re.sub('[^a-zA-Z]', ' ', df_title['title'][i]) #Removendo caracteres e pontuação           
  review = review.lower()   # Transformando as letras todas em minúsculo         
  review = review.split()   # Tokenizando pelo espaço entre palavras        
  ps = PorterStemmer()      # Normalizando as palavras pelo radical      
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]  #Eliminando Stopwords          
  review = ' '.join(review) # Juntando as palavras        
  corpus.append(review)     # Salvando as palavras no Corpus

In [ ]:
# Printado a tabela 
dftitle = pd.DataFrame({'title':corpus})

###**Pré-Processamento Tradicional**

![](https://blog.mackenzie.br/content/images/2020/09/giphy-1.gif)

In [ ]:
#Transformando palavras em números
from sklearn.preprocessing import LabelEncoder
dfcategory = df_category.apply(LabelEncoder().fit_transform)

In [ ]:
# Criando novo dataframe 
df_new = pd.concat([df_link, dftitle, dfcategory], axis=1)


In [ ]:
df_new

In [ ]:
# Fazendo a contagem das palavras
from sklearn.feature_extraction.text import CountVectorizer   
cv = CountVectorizer(max_features = 1500) 
X = cv.fit_transform(corpus).toarray() 
y = df_new.iloc[:, 2].values

In [ ]:
X

In [ ]:
y

In [ ]:
# Amostrando os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

#4° Construíndo a Máquina Preditiva 


### Machine Learning


In [ ]:
#Criando a Máquina Preditiva
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

#5° Avaliação da Máquina Preditiva


In [ ]:
#Avaliando a Máquina Preditiva
classifier.score(X_test, y_test)
from sklearn.metrics import classification_report
 print(classification_report(y_test, y_pred))